# DBLAB 과제

1. 공유받은 데이터를 이용하여 특징벡터로 변환
    1. 한줄당 한개의 특징벡터
    2. 모든 벡터의 크기는 같아야한다. 차원수
    3. Skobert를 이용하여 만들것 (라이브러리)
2. Faiss의 HNSWFlat, IVFPQ, IFVFFlat, IndexLSH
3. 색인, 리뷰 입력하여 가까운 리뷰탐색

In [1]:
import polars as pl
import torch
from sentence_transformers import SentenceTransformer
import numpy as np
from torch.nn import DataParallel
import faiss
import time

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
all_path = "data/ratings.txt"

all_lazy = pl.scan_csv(
    all_path,
    separator="\t",
    has_header=True,
    infer_schema_length=4000,
    encoding="utf8-lossy",
)

all_clean = (
    all_lazy
    .drop_nulls(["document"])
    .with_columns([
        pl.col("document").str.strip_chars().alias("document")
    ])
)

all_df = all_clean.collect(engine="gpu")

print(all_df.head())
print(all_df.shape)

shape: (5, 3)
┌──────────┬─────────────────────────────────┬───────┐
│ id       ┆ document                        ┆ label │
│ ---      ┆ ---                             ┆ ---   │
│ i64      ┆ str                             ┆ i64   │
╞══════════╪═════════════════════════════════╪═══════╡
│ 8112052  ┆ 어릴때보고 지금다시봐도         ┆ 1     │
│          ┆ 재밌어요ㅋㅋ                    ┆       │
│ 8132799  ┆ 디자인을 배우는 학생으로,       ┆ 1     │
│          ┆ 외국디자이너와 그들이 일군 …    ┆       │
│ 4655635  ┆ 폴리스스토리 시리즈는 1부터     ┆ 1     │
│          ┆ 뉴까지 버릴께 하나도 없음…      ┆       │
│ 9251303  ┆ 와.. 연기가 진짜 개쩔구나..     ┆ 1     │
│          ┆ 지루할거라고 생각했는데…        ┆       │
│ 10067386 ┆ 안개 자욱한 밤하늘에 떠 있는    ┆ 1     │
│          ┆ 초승달 같은 영화.               ┆       │
└──────────┴─────────────────────────────────┴───────┘
(199992, 3)


In [3]:
MODEL_NAME = "jhgan/ko-sbert-multitask"

device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer(MODEL_NAME, device=device)


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
texts = all_df["document"].to_list()

embeddings = model.encode(
    texts,
    batch_size=2048,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True,
).astype("float32")

emb_list = [v.tolist() for v in embeddings]

train_with_emb = all_df.with_columns(
    pl.Series("embedding", emb_list)
)

train_with_emb.write_parquet("train_with_sbert_embedding.parquet")

Batches: 100%|██████████| 98/98 [01:29<00:00,  1.09it/s]


In [5]:
df_check = pl.read_parquet("train_with_sbert_embedding.parquet")
print(df_check)
print(df_check["embedding"][0][:5])
print(len(df_check["embedding"][0]))


shape: (199_992, 4)
┌──────────┬─────────────────────────────────┬───────┬─────────────────────────────────┐
│ id       ┆ document                        ┆ label ┆ embedding                       │
│ ---      ┆ ---                             ┆ ---   ┆ ---                             │
│ i64      ┆ str                             ┆ i64   ┆ list[f64]                       │
╞══════════╪═════════════════════════════════╪═══════╪═════════════════════════════════╡
│ 8112052  ┆ 어릴때보고 지금다시봐도         ┆ 1     ┆ [0.020148, 0.002106, … 0.03217… │
│          ┆ 재밌어요ㅋㅋ                    ┆       ┆                                 │
│ 8132799  ┆ 디자인을 배우는 학생으로,       ┆ 1     ┆ [0.0021, 0.043697, … 0.091742]  │
│          ┆ 외국디자이너와 그들이 일군 …    ┆       ┆                                 │
│ 4655635  ┆ 폴리스스토리 시리즈는 1부터     ┆ 1     ┆ [-0.003628, 0.06392, … 0.06888… │
│          ┆ 뉴까지 버릴께 하나도 없음…      ┆       ┆                                 │
│ 9251303  ┆ 와.. 연기가 진짜 개쩔구나..     ┆ 1     ┆ [0.045442, 0.018689

In [6]:
PARQUET_PATH = "train_with_sbert_embedding.parquet"
EMB_COL = "embedding"
ID_COL = "id"
DOC_COL = "document"
LABEL_COL = "label"

USE_GPU = True
GPU_ID = 0

In [7]:
df = pl.read_parquet(PARQUET_PATH)

emb_list = df[EMB_COL].to_list()
embeddings = np.vstack(emb_list).astype("float32")   # (N, d)

# 메타 정보 (id, document, label)
ids = df[ID_COL].to_numpy() if ID_COL in df.columns else None
docs = df[DOC_COL].to_list() if DOC_COL in df.columns else None
labels = df[LABEL_COL].to_numpy() if LABEL_COL in df.columns else None

print("embeddings shape:", embeddings.shape)

embeddings shape: (199992, 768)


In [9]:
print("Faiss version:", faiss.__version__)
print("Num GPUs detected by Faiss:", faiss.get_num_gpus())

Faiss version: 1.7.2
Num GPUs detected by Faiss: 2


In [12]:
def build_multi_gpu_flat_index(embeddings):
    emb = embeddings.astype("float32").copy()
    faiss.normalize_L2(emb)

    d = emb.shape[1]

    cpu_index = faiss.IndexFlatIP(d)
    cpu_index.add(emb)

    # GPU 전체에 샤딩
    gpu_index = faiss.index_cpu_to_all_gpus(cpu_index)

    return gpu_index


In [23]:
def build_ivfflat_multi_gpu_index(
    embeddings: np.ndarray,
    nlist: int = 1024,
    training_samples: int = 100_000,
):
    """
    IVFFlat CPU 인덱스를 만든 뒤, 모든 GPU에 샤딩해서 멀티-GPU 인덱스를 생성.
    코사인 유사도: L2 정규화 + Inner Product.
    """
    # 1) float32 + 복사본
    emb = np.asarray(embeddings, dtype="float32").copy()
    faiss.normalize_L2(emb)

    n, d = emb.shape
    print(f"[IVFFlat] embeddings: n={n}, d={d}")

    # 2) CPU IVFFlat 인덱스 (quantizer = FlatIP)
    quantizer = faiss.IndexFlatIP(d)
    cpu_index = faiss.IndexIVFFlat(
        quantizer,
        d,
        nlist,
        faiss.METRIC_INNER_PRODUCT,
    )

    # 3) train (일부 샘플 사용)
    train_size = min(training_samples, n)
    train_idx = np.random.choice(n, size=train_size, replace=False)
    train_vecs = emb[train_idx]

    print(f"[IVFFlat] training with {train_vecs.shape[0]} vectors ...")
    t0 = time.perf_counter()
    cpu_index.train(train_vecs)
    t1 = time.perf_counter()
    print(f"[IVFFlat] train done in {t1 - t0:.3f}s")

    # 4) 전체 벡터 add
    t0 = time.perf_counter()
    cpu_index.add(emb)
    t1 = time.perf_counter()
    print(f"[IVFFlat] add {n} vectors done in {t1 - t0:.3f}s")

    print(f"[IVFFlat] cpu_index.ntotal = {cpu_index.ntotal}")

    # 5) 멀티 GPU로 샤딩 (모든 GPU 사용)
    #    - index_cpu_to_all_gpus는 IVF 계열에서도 잘 작동한다.
    print("[IVFFlat] sharding index to all GPUs ...")
    t0 = time.perf_counter()
    gpu_index = faiss.index_cpu_to_all_gpus(cpu_index)
    t1 = time.perf_counter()
    print(f"[IVFFlat] index_cpu_to_all_gpus done in {t1 - t0:.3f}s")

    # 6) 검색 정확도/속도용 nprobe 설정 (기본 10 정도에서 시작해서 튜닝)
    gpu_index.nprobe = min(16, nlist)   # 예: 16
    print(f"[IVFFlat] nprobe = {gpu_index.nprobe}")

    return gpu_index


In [14]:
def build_ivfpq_multi_gpu_index(
    embeddings: np.ndarray,
    nlist: int = 1024,         # IVF centroid 개수
    m: int = 64,               # subquantizer 개수 (d % m == 0 권장)
    nbits: int = 8,            # 각 subvector를 몇 bit로 양자화할지 (8이면 256 codeword)
    training_samples: int = 100_000,
):
    """
    IVFPQ CPU 인덱스를 만든 뒤, 모든 GPU에 샤딩해서 멀티-GPU 인덱스를 생성.
    코사인 유사도: L2 정규화 + Inner Product.
    """
    # 1) float32 + 복사본
    emb = np.asarray(embeddings, dtype="float32").copy()
    faiss.normalize_L2(emb)

    n, d = emb.shape
    print(f"[IVFPQ] embeddings: n={n}, d={d}")

    if d % m != 0:
        print(f"[IVFPQ] WARNING: dim={d} is not divisible by m={m}. "
              "Faiss는 동작하긴 하지만, 보통 d % m == 0 이 되게 설정하는 걸 권장합니다.")

    # 2) CPU IVFPQ 인덱스 (quantizer = FlatIP)
    quantizer = faiss.IndexFlatIP(d)
    cpu_index = faiss.IndexIVFPQ(
        quantizer,
        d,
        nlist,
        m,
        nbits,
        faiss.METRIC_INNER_PRODUCT,
    )

    # 3) train (일부 샘플 사용)
    train_size = min(training_samples, n)
    train_idx = np.random.choice(n, size=train_size, replace=False)
    train_vecs = emb[train_idx]

    print(f"[IVFPQ] training with {train_vecs.shape[0]} vectors ...")
    t0 = time.perf_counter()
    cpu_index.train(train_vecs)
    t1 = time.perf_counter()
    print(f"[IVFPQ] train done in {t1 - t0:.3f}s")

    # 4) 전체 벡터 add
    t0 = time.perf_counter()
    cpu_index.add(emb)
    t1 = time.perf_counter()
    print(f"[IVFPQ] add {n} vectors done in {t1 - t0:.3f}s")

    print(f"[IVFPQ] cpu_index.ntotal = {cpu_index.ntotal}")

    # 5) 멀티 GPU로 샤딩 (모든 GPU 사용)
    print("[IVFPQ] sharding index to all GPUs ...")
    t0 = time.perf_counter()
    gpu_index = faiss.index_cpu_to_all_gpus(cpu_index)
    t1 = time.perf_counter()
    print(f"[IVFPQ] index_cpu_to_all_gpus done in {t1 - t0:.3f}s")

    # 6) 검색 정확도/속도용 nprobe 설정
    gpu_index.nprobe = min(16, nlist)   # 시작은 8~16 정도에서, 나중에 튜닝
    print(f"[IVFPQ] nprobe = {gpu_index.nprobe}")

    return gpu_index



In [ ]:
def build_hnsw_index(
    embeddings: np.ndarray,
    M: int = 32,
    ef_construction: int = 200,
):
    emb = np.asarray(embeddings, dtype="float32").copy()
    faiss.normalize_L2(emb)          # 코사인용 정규화

    n, d = emb.shape
    print(f"[HNSW] embeddings: n={n}, d={d}")

    index = faiss.IndexHNSWFlat(d, M, faiss.METRIC_INNER_PRODUCT)
    index.hnsw.efConstruction = ef_construction

    t0 = time.perf_counter()
    index.add(emb)
    t1 = time.perf_counter()
    print(f"[HNSW] add {n} vectors done in {t1 - t0:.3f}s")
    print(f"[HNSW] ntotal = {index.ntotal}")

    return index


In [ ]:
def encode_query(text: str) -> np.ndarray:
    q_emb = model.encode(
        [text],
        convert_to_numpy=True,
        normalize_embeddings=True,   # 코사인 유사도용
    ).astype("float32")
    return q_emb

In [ ]:
def search_similar_reviews(
    query_text: str,
    top_k: int = 5,
    index=None,
    docs=None,
    labels=None,
):
    assert index is not None, "Faiss index (gpu_index)가 필요합니다."

    q_emb = encode_query(query_text)

    D, I = index.search(q_emb, top_k)
    nn_indices = I[0]
    nn_scores = D[0]

    results = []
    for rank, (idx, score) in enumerate(zip(nn_indices, nn_scores), start=1):
        item = {
            "rank": rank,
            "index": int(idx),
            "score": float(score),
        }
        if labels is not None:
            item["label"] = int(labels[idx])
        if docs is not None:
            item["text"] = docs[idx]
        results.append(item)

    return results


In [ ]:
def print_similar_reviews(
    query_text: str,
    top_k: int = 5,
    index=None,
    docs=None,
    labels=None,
):
    results = search_similar_reviews(
        query_text=query_text,
        top_k=top_k,
        index=index,
        docs=docs,
        labels=labels,
    )

    print("====[Query]====")
    print(f'[{query_text}]')
    print()

    print(f"[Top {top_k} nearest reviews]")
    for r in results:
        line = f"{r['rank']}. idx={r['index']}, score={r['score']:.4f}"
        if "label" in r:
            line += f", label={r['label']}"
        print(line)
        if "text" in r:
            print(r["text"])
        print("-" * 80)


In [12]:
# hnsw index
query = "연기도 좋고 감동적인 영화였어요"
print_similar_reviews(
    query_text=query,
    top_k=5,
    index=hnsw_index,
    docs=docs,
    labels=labels,
)

[Query]
연기도 좋고 감동적인 영화였어요

[Top 5 nearest reviews]
1. idx=41293, score=0.9078, label=1
재미있고, 뭔가 남는 영화같아서 좋았습니다... 연기도 좋았고.. 잘만들어진 영화라 느낍니다.
--------------------------------------------------------------------------------
2. idx=79936, score=0.9074, label=1
재미있고 감동도 준 좋은 영화였다.
--------------------------------------------------------------------------------
3. idx=7874, score=0.8978, label=1
재미있고 감동도 있는 영화였습니다
--------------------------------------------------------------------------------
4. idx=101235, score=0.8973, label=1
너무 좋았다. 좋은 영화다
--------------------------------------------------------------------------------
5. idx=78246, score=0.8960, label=1
좋고 재미있는 영화였다.
--------------------------------------------------------------------------------


In [ ]:
# GPU MEM 초기화
import gc
gc.collect()

import torch
torch.cuda.empty_cache()

In [38]:
query = "영화 내용이 너무 지루했어요. 배우들 연기도 안좋았어요."

for i in range(0,4):
    print("=" * 80)
    if i == 0:
        print("====[flat_index START]====")
        gpu_index = build_multi_gpu_flat_index(embeddings)
        print("====[flat_index DONE]====")
        print("=" * 80)
        print()
        index = gpu_index
    elif i == 1:
        print("====[ivfflat_index START]====")
        ivf_gpu_index = build_ivfflat_multi_gpu_index(embeddings, nlist=1024, training_samples=100_000)
        print("====[ivfflat_index DONE]====")
        print("=" * 80)
        print()
        index = ivf_gpu_index
    elif i == 2:
        print("====[ivfpq_gpu_index START]====")
        ivfpq_gpu_index = build_ivfpq_multi_gpu_index(
            embeddings,
            nlist=1024,
            m=32,
            nbits=8,
            training_samples=100_000,
        )
        print("====[ivfpq_gpu_index DONE]====")
        print("=" * 80)
        print()
        index = ivfpq_gpu_index
    elif i == 3:
        print("====[hnsw_index START]====")
        hnsw_index = build_hnsw_index(embeddings, M=32, ef_construction=200)
        print("====[hnsw_index DONE]====")
        print("=" * 80)
        print()
        index = hnsw_index
    else :
        print("Not A Model")
    print_similar_reviews(
        query_text=query,
        top_k=5,
        index=index,
        docs=docs,
        labels=labels,
    )
    print()
    del index
import gc
gc.collect()

import torch
torch.cuda.empty_cache()


====[flat_index START]====
====[flat_index DONE]====

[Query]
영화 내용이 너무 지루했어요. 배우들 연기도 안좋았어요.

[Top 5 nearest reviews]
1. idx=173678, score=0.8729, label=0
영화 전개가 너무 지루해요.
--------------------------------------------------------------------------------
2. idx=154357, score=0.8659, label=0
배우들만 고생한 영화내용들이 너무 재미없엇다ㅠ
--------------------------------------------------------------------------------
3. idx=100915, score=0.8577, label=0
영화로서 너무 지루하다.
--------------------------------------------------------------------------------
4. idx=180772, score=0.8500, label=0
초반엔 좋았지만. 갈수록 지루하다. 배우들 연기력도 어색. 보는내내 지루하고 진부한 초딩같은 영화...
--------------------------------------------------------------------------------
5. idx=136915, score=0.8467, label=0
너무 지루한 영화였다...
--------------------------------------------------------------------------------

====[ivfflat_index START]====
[IVFFlat] embeddings: n=199992, d=768
[IVFFlat] training with 100000 vectors ...
[IVFFlat] train done in 10.187s
[IVFFlat] add 1999